# PROJET FINAL MACHINE LEARNING

## REPONSE AUX QUESTIONS

**1-   Quelles sont les définitions de valeurs estimées et de valeurs prédites ?**

-  Les valeurs estimées sont généralement des valeurs calculées à partir d'un modèle statistique qui a été ajusté aux données observées et qui sont souvent des résultats d'une estimation basée sur un ensemble de paramètres déterminés par le modèle.
-   Les valeurs prédites sont des valeurs calculées par un modèle pour de nouvelles données, pour la plupart du temps distinctes de celles utilisées pour ajuster le modèle. Pour ce faire, on évalue généralement ses performances en utilisant un ensemble de données de test après avoir entraîné un modèle sur un ensemble de données d'apprentissage.

**2-  Pourquoi quand on compare des méthodes doit on comparer des méthodes optimales ?**

La comparaison des méthodes optimales assure que chaque méthode est évaluée dans son meilleur contexte possible. Cela garantit une comparaison équitable où chaque méthode a la possibilité de montrer son meilleur rendement. C'est-à-dire qu'on aura des informations que la performance maximale que chaque méthode peut atteindre.

**3-    Démarche à suivre pour la réalisation de ce projet:**

-   Importation du jeu de données
-   Analyse exploratoire du jeu de données
-   Nettoyage si necessaire du jeu de données afin qu'il soit optimisé à nos codes
-   Sélection des différents algorithmes de machine learning
-   Définition des grilles des différents paramètres associés aux algorithmes
-   Faire une première validation croisée afin de trouver les paramètres optimaux de chaque modèle
-   Faire une deuxième validation croisée pour maintenant mesurer la performance de chaque modèle
-   Faire de la feature engineering pour mesurer la performance des modèles

## REALISATION DU PROJET

**IMPORTATION DES BIBLIOTHÈQUES**

In [1]:
import pandas as pd
import numpy as np
from summarytools import dfSummary
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

**IMPORTATION DE NOS DONNEES**

In [2]:
df = pd.read_csv('vin.csv', sep=',')
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,mauvais
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,mauvais
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,mauvais
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,bon
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,mauvais


**ANALYSE ET NETTOYAGE DES DONNEES**

In [3]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,fixed.acidity[float64],Mean (sd) : 8.3 (1.7)min < med < max:4.6 < 7.9 < 15.9IQR (CV) : 2.1 (4.8),96 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACnUlEQVR4nO3du47aQBSA4TOwIHORLUQEHU3KSCkoeYg8bB4hSDwBbSo6JAsZ4WBhjFNtmrBcxuNljvV//SxT/LJ2ZB+NKctSAN+1Xr0B4BFvr97ALcaYnoh0LZefyrI8utwPXsfbUI0xvfF4/CMMw5HN+v1+vzPG/CTWZvA2VBHphmE4WiwWxyiKsmcWJkkSrFarURzHXREh1AbwOVQREYmiKJtMJn8slvacbwYvw2EKKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqeP+Fv62iKDoiEhpjbP8Ew4EeaWSoaZp28jz/PpvN2u12+6l5q3cMB/qlkaGeTqe3IAj68/k8m06nu2fXMxzon0aG+m4wGNgOBoowHOgVDlNQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSrU+plfxet3wsvl0nG5H+hVW6hVr9/J8zwoiuJblmW/RMT2m1I0RJ1PVOvrd0RENpvNaL1e98/nc7uOzUGX2r/wt71+Z7fb8YU9/uEwBRUaPTNVRcVxa0atHSPUK6qOWzNq7R6hXlFl3JpR63oQ6g0Vxq05CDrGYQoqECpUIFSoQKhQgVChAqf+GvCywD1CdYyXBfUgVMd4WVAPQq0JLwvc4jAFFXiieoZLMq4jVI9wScbHCNUjXJLxsbuhVpgkZYrUEpdk/O9mqFUmSZkihUv3nqjWk6RMkcKlh/5HtZkkZYr0NZr6+pbDVIM0+fUtoTZI1de3y+VyEsfxF2PM3nILLRG5WK69+TR/KNQkSYJnf/VwOAQiImmaBtvttv+Z67X+tqt92zgej5WexkVRdNI0/TocDn+3Wq382fX3nuZ/AbiTgEtJ+TsoAAAAAElFTkSuQmCC"">",0(0.0%)
2,volatile.acidity[float64],Mean (sd) : 0.5 (0.2)min < med < max:0.1 < 0.5 < 1.6IQR (CV) : 0.2 (2.9),143 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACiUlEQVR4nO3dTY6iQBiA4a/GP/wJxJjozgtMMguXHGIOO0dw4Qm8gTsTQzAyzYjArJx0Mj3dQhUjH/0++0IWb+iiinSZsiwFaLsvz74B4BH9Z99AU4wxYxEZWlziWpbli6v7gZ1OhmqMGS8Wi+++78/rXuN8PkfGmB/E2g6dDFVEhr7vz8MwfAmCIK06OI5jb7fbzU+n01BECLUFuhqqiIgEQZAul8ufNYePnd4MrPAyBRUIFSoQKlQgVKjQ6pcpi7VQvyiKgev7wfO0NlSbtdAsy7w8z7+maboVkbpv/WiRRkO13B3yZ7PZMgzDc9W10MPhMN/v95Pb7dar+dtomcZCtd0duj8VR6PRtupaaBRFrIF2TJNPVKvdIZ6KeK3xOWrd3SGeiniN5SmoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVGjtYRPPluf5QER8Y0yd4ZxM7RihviFJkkGWZd/W63Wv1+tV/rfunEztHqG+4Xq99j3Pm2w2m3S1WkVVxnIydTMI9R3T6bTu6dScP+AYL1NQgVChAqFCBUKFCoQKFQgVKhAqVPhwHdXiKHO/KIpBjXHAX94N1eYo8/sx5mmabkWkzqI58MdHT9TaR5lzjDlcemgLtc5R5p/5GHO+vHKPvX7H+PKqGYTqGF9eNYNQG8KXV24RastYzm9FOjrHJdQWsZ3finR3jkuoLWIzvxXp9hz3oVDjOPaqXvhyuXgiIkmSeMfjcfI/x2v97ftYG3me98Vu6vAsv8qy/Odfkd+3nScJ/aVy7AAAAABJRU5ErkJggg=="">",0(0.0%)
3,citric.acid[float64],Mean (sd) : 0.3 (0.2)min < med < max:0.0 < 0.3 < 1.0IQR (CV) : 0.3 (1.4),80 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACgUlEQVR4nO3cMY7aQBSA4TdhIQNENogEOlqKSCkoOUQOmyOkoKXhBnRIBhnh4MUYp4gSrRR2wTZe5ln/V+/YFD+7GvvNmizLBHDdh0d/AOAWT9d+wBjTFpFWwesfsyw7FFwL/PNmqMaY9mAw+O55Xr/IxXe73dYY84NYUda136gtz/P6s9ns4Pt+nOfCYRja+XzeD4KgJSKEilKu/ukXEfF9Px4Oh78KXL9dYA3wHzZTUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUKFm971PwojhvjL2VAZMcRLzoYqjBjiBZdDFRFGDPEHmymoQKhQgVChAqFCBUKFCs7v+h+Flw1uIdQLeNngHkK9jJcNjiHUN/CywR1spqACoUIFQoUKhAoVCBUqVLrrT9O0KSKeMabIcu98Pjc13hv3V1moURQ1kyT5Nh6PG41GI9ezSBGRJElsmqZf4zj+KSK5HhE98t6oRmWhHo/HJ2ttZzqdxqPRaJt3/Wq16i+Xy87pdGpoujeqUfkD/263W+ih+Xa7Lf3Q/JH3xn2xmYIKhAoVCBUqECpUIFSoQKhQgVChAqFCBSb8K1ByzoCDgRcQ6p2VnTPgYOBlhHpnZeYMOBj4OkKtSNE5A+Fg4EVspqACoUIFQoUKhAoVCBUqECpUIFSoQKhQgQf+jik5JyBS01kBQnVI2TkBkfrOChCqQ8r+P4I6zwoQqoNKzAmI1HRWgM0UVCBUqECoUIFQoQKbqZqp63ktQq2ROp/XuinUMAxt3gvv93srIhJFkV2v1533XK/13mU/92az+WSt7UwmE9Pr9Z7zrD0cDh8Xi8WXIAg+G2N2ee99B89Zlr365foNRqpjgXEhTmwAAAAASUVORK5CYII="">",0(0.0%)
4,residual.sugar[float64],Mean (sd) : 2.5 (1.4)min < med < max:0.9 < 2.2 < 15.5IQR (CV) : 0.7 (1.8),91 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG

La variable cible identifié ici est **'quality'**. Ainsi, notre tâche consistera à déterminer l'algorithme qui prédira au mieux la classe des individus. Après analyse de nos données, nous constatons que nos données sont assez équilibrées(*53.5% pour 'bon' et 46.5% pour mauvais*) alors, un rééquilibrage des données ne sera pas nécessaire. On aura donc qu'à binariser notre variable cible.

In [4]:
df["quality"] = (df["quality"] == "bon").astype(int)
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


Notre variable cible étant transtypée sous forme binaire, nous allons maintenant séparer nos variables explicatives de notre variable cible. Aussi, nous allons mettre de variables à échelle en nous servant de la fonction `StandardScaler()`

In [5]:
Y = df["quality"].values
X = df.drop(["quality"], axis=1).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**DEFINITION DE NOS MODELES DE CLASSIFICATION ET DE NOS GRILLES DE PARAMETRE**

In [6]:
models_classification = [
    ("Regression Logistique", LogisticRegression(max_iter=1000), {'C':[0.001, 0.01, 0.1, 1, 10, 100]}),
    ("Arbre de décision", DecisionTreeClassifier(), {'max_depth': [1, 5, 10, 15, 20]}),
    ("Bagging KPPV", BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=10)), {'n_estimators': [50, 100, 150, 200, 300]}),
    ("Random Forest", RandomForestClassifier(), {'n_estimators': [10, 50, 100, 150,200], 'max_depth':[2,3,4,6,8]}),
    ("XG Boost", XGBClassifier(), {'n_estimators': [50, 100, 150, 200, 300], 'max_depth': [3, 4, 5, 6, 7]}),
    ("ADA Boost", AdaBoostClassifier(), {'n_estimators': [50, 100, 150, 200, 300], 'learning_rate': [0.01, 0.1, 0.5, 1.0]}),
    ("Gradient Boosting", GradientBoostingClassifier(), {'n_estimators': [50, 100, 150, 200, 300], 'max_depth': [3, 4, 5, 6, 7],}),
    ("KPPV", KNeighborsClassifier(), {'n_neighbors': [1, 3, 5, 7, 10]})
]

**DEFINITION DE LA FONCTION D'EVALUATION DES DIFFERENTS MODELES**

In [7]:
def evaluation_model(X, Y, model, param_grid, num_folds=5):
    # Utilisation de KFold pour la validation croisée
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=1234)

    # Utilisation de GridSearchCV pour rechercher les meilleurs parametres avec la validation croisée (Première validation croisée)
    grid_search = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', cv=kfold)
    grid_search.fit(X, Y)

    best_model = grid_search.best_estimator_ # Meilleurs parametres

    # Calcul de l'Accuracy moyen (Deuxième validation croisée)
    accuracy_scores = cross_val_score(best_model, X, Y, scoring='accuracy', cv=kfold)
    average_accuracy = accuracy_scores.mean()

    return average_accuracy, best_model

**APPLICATION DE NOTRE FONCTION SUR NOS DONNEES**

In [8]:
for name, model, param_grid in models_classification:
    acc, best_model = evaluation_model(X_scaled, Y, model, param_grid)
    
    print(f"Modele: {name}")
    
    # Récupérer les hyperparamètres spécifiés dans models_classification
    params_specifie = {param: best_model.get_params()[param] for param in param_grid.keys()}
    
    print(f"Meilleurs Hyperparametres: {params_specifie}")
    print(f"Accuracy: {acc}")
    print("\n")


Modele: Regression Logistique
Meilleurs Hyperparametres: {'C': 0.01}
Accuracy: 0.7448452194357367


Modele: Arbre de décision
Meilleurs Hyperparametres: {'max_depth': 15}
Accuracy: 0.7373315047021944


Modele: Bagging KPPV
Meilleurs Hyperparametres: {'n_estimators': 300}
Accuracy: 0.7310873824451412


Modele: Random Forest
Meilleurs Hyperparametres: {'n_estimators': 150, 'max_depth': 8}
Accuracy: 0.7948667711598747


Modele: XG Boost
Meilleurs Hyperparametres: {'n_estimators': 200, 'max_depth': 7}
Accuracy: 0.8042515673981191


Modele: ADA Boost
Meilleurs Hyperparametres: {'n_estimators': 200, 'learning_rate': 0.1}
Accuracy: 0.7629878526645768


Modele: Gradient Boosting
Meilleurs Hyperparametres: {'n_estimators': 50, 'max_depth': 7}
Accuracy: 0.804233934169279


Modele: KPPV
Meilleurs Hyperparametres: {'n_neighbors': 1}
Accuracy: 0.7479839341692791




Ici, nous avons pour chaque modèle les valeurs optimales des paramètres pour les grilles définies, ainsi que leur précision. On remarque ici que le **XG Boost** et le **Gradient Boosting** sont plus précis avec une précision sensiblement égale à **80%**. Ces derniers sont suivis par le **Random Forest** avec une précision d'environ **79.92%**.

**FEATURE ENGINEERING**

In [9]:
X2 = np.concatenate((X_scaled,np.square(X_scaled)), axis=1)
X3 = np.concatenate((X2,np.power(X_scaled,3)), axis=1)

- *Comparaison des modèles sur la matrice X2:*

In [10]:
for name, model, param_grid in models_classification:
    acc, best_model = evaluation_model(X2, Y, model, param_grid)
    
    print(f"Modele: {name}")
    
    params_specifie = {param: best_model.get_params()[param] for param in param_grid.keys()}
    
    print(f"Meilleurs Hyperparametres: {params_specifie}")
    print(f"Accuracy: {acc}")
    print("\n")

Modele: Regression Logistique
Meilleurs Hyperparametres: {'C': 0.1}
Accuracy: 0.7485854231974921


Modele: Arbre de décision
Meilleurs Hyperparametres: {'max_depth': 20}
Accuracy: 0.7542065047021944


Modele: Bagging KPPV
Meilleurs Hyperparametres: {'n_estimators': 100}
Accuracy: 0.711065830721003


Modele: Random Forest
Meilleurs Hyperparametres: {'n_estimators': 150, 'max_depth': 8}
Accuracy: 0.8023922413793103


Modele: XG Boost
Meilleurs Hyperparametres: {'n_estimators': 50, 'max_depth': 6}
Accuracy: 0.81298197492163


Modele: ADA Boost
Meilleurs Hyperparametres: {'n_estimators': 300, 'learning_rate': 0.1}
Accuracy: 0.7661030564263323


Modele: Gradient Boosting
Meilleurs Hyperparametres: {'n_estimators': 200, 'max_depth': 4}
Accuracy: 0.7967417711598745


Modele: KPPV
Meilleurs Hyperparametres: {'n_neighbors': 1}
Accuracy: 0.7435834639498433




Après avoir ajouté nos variables élevées au carrée à nos variables initiales, nous avons pour chaque modèle les valeurs optimales des paramètres pour les grilles définies, ainsi que leur précision. On remarque ici que le **XG Boost** et le **Gradient Boosting** sont toujours plus précis, cette fois avec une précision sensiblement égale à **81%**(**Hausse de 1%**). Ces derniers sont suivis par le **Random Forest** avec une précision d'environ **79.54%**.

- *Comparaison des modèles sur la matrice X3:*

In [11]:
for name, model, param_grid in models_classification:
    acc, best_model = evaluation_model(X3, Y, model, param_grid)
    
    print(f"Modele: {name}")
    
    params_specifie = {param: best_model.get_params()[param] for param in param_grid.keys()}
    
    print(f"Meilleurs Hyperparametres: {params_specifie}")
    print(f"Accuracy: {acc}")
    print("\n")

Modele: Regression Logistique
Meilleurs Hyperparametres: {'C': 0.1}
Accuracy: 0.749212382445141


Modele: Arbre de décision
Meilleurs Hyperparametres: {'max_depth': 10}
Accuracy: 0.7542025862068965


Modele: Bagging KPPV
Meilleurs Hyperparametres: {'n_estimators': 300}
Accuracy: 0.7073060344827586


Modele: Random Forest
Meilleurs Hyperparametres: {'n_estimators': 100, 'max_depth': 8}
Accuracy: 0.7948765673981191


Modele: XG Boost
Meilleurs Hyperparametres: {'n_estimators': 50, 'max_depth': 6}
Accuracy: 0.81298197492163


Modele: ADA Boost
Meilleurs Hyperparametres: {'n_estimators': 300, 'learning_rate': 0.1}
Accuracy: 0.7661030564263323


Modele: Gradient Boosting
Meilleurs Hyperparametres: {'n_estimators': 150, 'max_depth': 5}
Accuracy: 0.8048608934169279


Modele: KPPV
Meilleurs Hyperparametres: {'n_neighbors': 1}
Accuracy: 0.7304565047021944




Après l'ajout de nos variables élevées au cube à notre jeu de donné `X2`, nous avons pour chaque modèle les valeurs optimales des paramètres pour les grilles définies, ainsi que leur précision. On remarque ici que le **XG Boost** et le **Gradient Boosting** sont toujours les plus précis(**81%** pour le *XG Boost* et **80%** pour le *Gradient Boosting*). Ces derniers sont suivis par le **Random Forest** avec une précision d'environ **79.79%**.